# Laserscanning - Exercise 3

#### Please upload the implemented solutions till <u>22.11.2022</u> to the studip folder of your group. The file should follow this format:
##### EX03_Group_XX.ipynb (e.g. EX03_Group_04.ipynb)
(In case you need to upload additional files, please also upload them using the same schema, e.g. EX03_Group_04.zip)

# Iterative Clostest Point (ICP)

ICP is an algorithm for the estimation of the transformation parameters that does not require known correspondences between the points. Given two sets of points `left` and `right` The goal of this assignment is to estimate:

- Find the correspondences between the points
- Estiamte the tranformation parameters between the two sets


## Approach

    1) Set approximate values for $R$, $s$, $t$ (here: only the rotation, as scale and translation are given)
    2) Assign points of the `left` and `right` point cloud to each other
    3) Compute the transformation using the correspondences from step 2
    4) Transform the left point cloud
    5) Compute mean squared error (MSE)
        - Iteration: Go to line 2. until convergence 
    6) Compute final transformation

### 1) Set approximate values

Translation and scale are given:

$s_0=1$

$ t_0=\begin{pmatrix}
    0.6 \\
    1.0 \\
    -0.2
    \end{pmatrix}$
    
The initial rotation matrix $R_0$ can be approcimated by visual inspection. You may use the provided `animation` function. Optionally you may optimize approximate values by a (randomized) grid search.

In [11]:
!pip install scikit-learn

In [12]:
import numpy as np

#from tools import rotmat_from_quat
from plyfile import PlyData
from sklearn.neighbors import KDTree

ModuleNotFoundError: No module named 'plyfile'

In [7]:
visualization = True  # Use this flag to deactive the visualization
if visualization:
    import ipyvolume as ipv

def animate(lefts_array, right):
    lefts_array = np.stack(lefts_array)
    ipv.figure()
   
    if len(lefts_array.shape) == 3 : 
        sl = ipv.scatter(lefts_array[:,:,0], lefts_array[:,:,1], lefts_array[:,:,2], size=1, marker='sphere',color='green', color_selected='white')
    elif len(lefts_array.shape) == 2:
        sl = ipv.scatter(lefts_array[:,0], lefts_array[:,1], lefts_array[:,2], size=1, marker='sphere',color='green', color_selected='white')
    else:
        raise UnkownError("Unsupported state.")
        
    ipv.scatter(right[:,0], right[:,1], right[:,2],  size=1, marker='sphere',color='red')
    ipv.animation_control(sl) # shows controls for animation controls
    ipv.show()


# load data
_data = PlyData.read('tango5cm.ply').elements[0].data
left = np.stack((_data['x'], _data['y'], _data['z']), axis=1)
_data = PlyData.read('riegl5cm.ply').elements[0].data
right = np.stack((_data['x'], _data['y'], _data['z']), axis=1) 

print("Left points count: {}".format(left.shape[0]))
print("Right points count: {}".format(right.shape[0]))

# example visualization
if visualization:
    animate((left), right)
    
    # The first paramter of the animate function can be a list of points list. 
    # In this animation is generated between the point lits. 
    # mv = np.array([4, 4, 4])
    # animate((left, left - mv, left + mv*0.5), right)

NameError: name 'PlyData' is not defined

### 2) Estimate Correspondences
We assume that the respective closest points (euclidean distance) in the 'left' and 'right' point cloud are the corrensponding ones. The usage of a KDTree data structure can improve the implementation in terms of computational speed. Please read this [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KDTree.html#sklearn.neighbors.KDTree.two_point_correlation).
Feel free to use another correspndence estimation strategie presented in the lecture. Ignore the automated test in that case.

In [8]:
def sort_by_corrspondence(left, right):
    # This method takes two lists of points and return a subset of the right list with following properties:
    # return right sublist with same length as left list and the points left[i] and right_sublist[i] are the closest points.
    ### BEGIN SOLUTION
    list=[]
    for i in range(len(left)):
        Y = right[i]
   #     print(Y.reshape(1,3))
    
        X = left  # 10 points in 3 dimensions
        tree = KDTree(X, leaf_size=3) 
        dist, ind = tree.query(Y.reshape(1,3),k=3)
       # ind =tree.two_point_correlation(X, Y)
  #      print(ind)  # indices of 3 closest neighbors
        list.append(right[ind[0][0]])


  #  print(list)  # distances to 3 closest neighbors

    return (list)
    ### END SOLUTION
    
    #raise NotImplementedError()

<u>Please explain your code here:</u>

SOME SENTENCES
We take the nearest points (Euclidean distance) in each of the "left" and "right" point clouds as the corrensponding ones. we make the "left" point cloud a KDTree data structure. then we traverse all the points in the "right" point cloud so that each point can find a match in the "left" point cloud. Finally we put all the matching points into the list and return.

In the following cell you can test your implementation and can compare it to correct resulting values.

In [ ]:
def test_cor():
    tpl = np.array([[0,0,1],
                    [0,1,0],
                    [1,0,0]])
    tpr = np.array([[0,0.5,-0.5],
                    [0,0,1.5],
                    [0.1,0,0]])
    sr = sort_by_corrspondence(tpl,tpr)
    assert all(tpr[0] == sr[1]) and all(tpr[1] == sr[0]) and all(tpr[2] == sr[2])
test_cor()

### 3) - 6) Implementation of the ICP
Implement a function for the ICP which takes the parameters `icp(s_0, t_0, R_0, left, right):` and returns `s, t, R,` the final estimated transformation parameters and a list of mean squared error for each iteration step. In order to implement this you might need to re-use some function from previous assignments.

In [ ]:
#Use this area to copy paste or import your code from earlier exercises
### BEGIN SOLUTION
from numpy import *;
import numpy as np;

def load_ply(filename):
    list1=[]
    list2=[]

    with open(filename, 'r') as ply_file: 
        for line in ply_file:
            line=line.strip('\n')
            line=line.split(' ')
           
            if '' in line:
                line.pop()
            else:
                line=line
            list1.append(line)
        list2=list1[list1.index(['end_header'])+1:]
        points=np.array(list2)
        points=points.astype(np.float64)
       
        return points
def sum_S(left_list, right_list):
    ### BEGIN SOLUTION
    
    s_x=left_list[0]*right_list[0]
    s_y=left_list[1]*right_list[1]
    s_z=left_list[2]*right_list[2]
    b=s_x+s_y+s_z

    return b

def compose_S(left, right):
    ### BEGIN SOLUTION
    s_xx=0
    s_yy=0
    s_zz=0
    s_xy=0
    s_xz=0
    s_yx=0
    s_yz=0
    s_zx=0
    s_zy=0
    for i in range(len(left)):
        m_xx=left[i][0]*right[i][0]
        m_yy=left[i][1]*right[i][1]
        m_zz=left[i][2]*right[i][2]
        m_xy=left[i][0]*right[i][1]
        m_xz=left[i][0]*right[i][2]
        m_yx=left[i][1]*right[i][0]
        m_yz=left[i][1]*right[i][2]
        m_zx=left[i][2]*right[i][0]
        m_zy=left[i][2]*right[i][1]
        
        s_xx+=m_xx
        s_yy+=m_yy
        s_zz+=m_zz
        s_xy+=m_xy
        s_xz+=m_xz
        s_yx+=m_yx
        s_yz+=m_yz
        s_zx+=m_zx
        s_zy+=m_zy
        pass

    S=array([[s_xx+s_yy+s_zz,s_yz-s_zy,s_zx-s_xz,s_xy-s_yx],
        [s_yz-s_zy,s_xx-s_yy-s_zz,s_xy+s_yx,s_zx+s_xz],
        [s_zx-s_xz,s_xy+s_yx,-s_xx+s_yy-s_zz,s_yz+s_zy],
        [s_xy-s_yx,s_zx+s_xz,s_yz+s_zy,-s_xx-s_yy+s_zz]])
    return S

def q_from_S(S):
    # take a look at numpy.linalg.eig() function for this task
    ### BEGIN SOLUTION
    
    c = np.linalg.eig(S)
    
    ew=c[0]
    l=list(ew)  
    m=max(l)
    num=l.index(m)
    d=c[1]
    q=d[:,num]

    return q    

def calc_scale(left, right):
    ### BEGIN SOLUTION
    import math as m
    r_prime=[]
    l_prime=[]
    l_com=np.mean(left,axis=0)
    r_com=np.mean(right,axis=0)
    for i in right:
        r_prime.append(np.power(np.linalg.norm(i-r_com),2))
    for j in left:
        l_prime.append(np.power(np.linalg.norm(j-l_com),2))
    r_sum=np.sum(r_prime)
    l_sum=np.sum(l_prime)
    s=m.sqrt(r_sum/l_sum)

    return s

def calc_t(l_com, r_com, s, R):
    ### BEGIN SOLUTION
    import numpy as np
    
    t=r_com-s*np.dot(R,l_com)
    

    return t
### END SOLUTION

#raise NotImplementedError()

In [ ]:
# implementation of the icp algorithm
def icp(s, t, R, left, right):
    left_states = [left]
    for i in range(len(left)):
        left_trans = s*np.dot(R, left[i].transpose()).transpose() + t
        left_states.append(left_trans) # if you store the intermidiate states of the left list, it will be animated.
    
    max_iter = 20
    mse = []
    error = 0
    ### BEGIN SOLUTION
    from tools import rotmat_from_quat
    l_com=np.mean(left,axis=0)
    r_com=np.mean(right,axis=0)
    for j in range(max_iter):
        sr = sort_by_corrspondence(left, right)
        S = compose_S(sr, right)
        q = q_from_S(S)
        R = rotmat_from_quat(q)
        s = calc_scale(left, right)
        t = calc_t(l_com, r_com, s, R)

        re = s*np.dot(R,left[j])+t

        er = np.power(np.linalg.norm(re-right[j]),2)
        mse.append(er)

        if error<er:
            error=er

            R = R
            s = s
            t = t
    
    ### END SOLUTION
    
    #raise NotImplementedError()
    
    if visualization:
        animate(left_states, right)
    
    return (s, R, t), mse

## Application

- Use the given 3D point clouds `tango5cm.ply` (left) and `riegl5cm.ply` (right, reference) as input data for ICP (downloadable from StudIP)
- Determine the final transformation parameters using your ICP implementation
- Plot the mean squared errors (MSE) of the ICP iterations
- Explain the whole procedure (ICP) and findings in your own words (last cell)!

In [9]:
# define your inital scale, translation (given at the beginning) and approximate rotation matrix

### BEGIN SOLUTION

left_ = load_ply('tango5cm.ply')
right_ = load_ply('riegl5cm.ply')
left=left_[:,0:3]
right=right_[:,0:3]
s = 1
t = np.array([ 0.6,   1.0,   -0.2])
R = np.array([[ 0,  -1,  0],
             [ 1,  0,  0],
             [ 0, 0, 1]])

### END SOLUTION

# raise NotImplementedError()

(s, t, R), mse = icp(s, t, R, left, right)

%matplotlib inline

import matplotlib.pyplot as plt

plt.plot(mse)

NameError: name 'load_ply' is not defined

<u>Please explain your code here:</u>

SOME SENTENCES



ICP is one of the widely used algorithms in aligning three dimensional models given an initial guess of the rigid transformation required.
The ICP algorithm steps are: For each point in the source point cloud, match the closest point in the reference point cloud. Estimate the combination of rotation and translation using a root mean square point to point distance metric minimization technique which will best align each source point to its match found in the previous step. This step may also involve weighting points and rejecting outliers prior to alignment.Transform the source points using the obtained transformation.Iterate (re-associate the points, and so on).